# ImportError: DLL load failed while importing cv2: The specified module could not be found.
> Fix common OpenCV python DLL load failed error when building python bindings from source

- toc: true 
- badges: true
- comments: true
- sticky_rank: 1
- author: cudawarped
- image: images/diagram.png
- categories: [opencv, cuda, python]

So your on Windows using python >= 3.8, you have built OpenCV >= 4.6 from source, manually copied .. python path them to your distributions site... directory, added the path to the directory containing opencv.dll and any other shared library dependencies and your seeing something similar to the following.

In [1]:
#import os
#os.environ["OpenCV_LOADER_DEBUG"]="1"
import cv2

ModuleNotFoundError: No module named 'cv2'

What's going on? Although the message is quite explicit regarding the cause, it doesn't really help with finding a solution. In a nutshell python has found .pyd (shared lib), tried and failed to load it because a it can't find a dependant shared library. Now the advice I have seen online regarding this is to dig out trusty dependancy walker, load the .pyd and see which dependencies the system can't find.

Now this is solid advice if we had a C++ application and/or we were using python < 3.8 (which uses the system/user path for dll re), however we are not, so even if dependency walker can't detect any problems we may still be facing the above error.

There is a missing dll, which needs to be added with ... before calling import cv2, now if you have installed the python loader with OpenCV then you will likeliy need to search with .... to find it, however if you have not you may be able to try the quick fix below may work for you


### Quick fix when not using the OpenCV python loader

Using the flag OPENCV_SKIP_PYTHON_LOADER=ON prevents the python bindings from being installed when you build install OpenCV, therefore depending on how you installed cv2 there shouldn't be a cv2 directory inside lib/site-sitepackages and you should something similar to the following

There are multiple reasons that the OpenCV python loader may be missing
1) You didn't build install

2) OPENCV_SKIP_PYTHON_LOADER=ON was passed to cmake

3) The lib/site-packages/cv2 diretory was deleted

Whatever the reason if you are seeing the above error you have likely manually "installed" the python bindings either with

1) set PYTHONPATH="OPENCV_BUILD_PATH/lib/python3/"

2) or by manually copying .... pyd to the lib/site-packages/

Whichever way this was performed, you will be missing (unless your using a distribution which automatically adds .. directories from the path to the dll search path)

os.add_dll_directory("OPENCV_BUID_PATH/bin/");

in this case there will be at least one dll search path which needs to be added.

In [24]:
#collapse-output
import os;
OPENCV_BUID_PATH = "D:/build/opencv/cuda_11_7_cc_8_6_ff_sym/"
os.add_dll_directory(OPENCV_BUID_PATH + "/bin/");
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

If that didn't work and you are using the CUDA modules you will also need to add 
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");, if that doesn't work then you need to locate the missing dll and manually add it, seee ...

In [25]:
#collapse-output
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

In [13]:
import sysconfig
import os
os.listdir(sysconfig.get_paths()["purelib"]+"/cv2")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\b\\mambaforge\\Lib\\site-packages/cv2'

In this case you will definitely be missing the directory containing .... and ... and if you built with CUDA you will also be missing ... therefore before doing anything else 

## You have installed the python loader

When you built OpenCV with cmake you didn't pass OPENCV_SKIP_PYTHON_LOADER=ON and there is a cv2 directory inside site-packages/lib

In [7]:
import sysconfig;
import os;
os.listdir(sysconfig.get_paths()["purelib"]+"/cv2")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\b\\mambaforge\\Lib\\site-packages/cv2'

The python loader script should have added the directory to ... and the CUDA SDK if you built with CUDA modules, so to track down the missing

## Built OpenCV with OPENCV_SKIP_PYTHON_LOADER=ON

Using the flag OPENCV_SKIP_PYTHON_LOADER=ON prevents the python bindings from being installed when you build install OpenCV, therefore there shouldn't be a cv2 directory insidde lib/site-sitepackages and you should see the following

In [9]:
import sysconfig;
import os;
os.listdir(sysconfig.get_paths()["purelib"]+"/cv2")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\b\\mambaforge\\Lib\\site-packages/cv2'

then you enter something similar to the below in the command prompt before 

set PYTHONPATH="OPENCV_BUILD_PATH/lib/python3/"

set PATH="OPENCV_BUID_PATH/bin";%PATH%

or for my set up in python after PYTHONPATH has been parsed

In [17]:
import os
OPENCV_BUID_PATH = "D:/build/opencv/cuda_11_7_cc_8_6_ff_sym/"

In [18]:
import sys
sys.path.append(OPENCV_BUID_PATH + "/lib/python3/")
sys.path.append(OPENCV_BUID_PATH + "/bin/")

Advice is to skip loader it is for stand alone distributions 
https://github.com/opencv/opencv/issues/13202#issuecomment-439730899
Add python path
Therefore the discussion assumes that the above is the method used for adding python etc.

In [19]:
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

What's going on?  Although the message is quite explicit regarding the cause, it doesn't really help with finding a solution.  In a nutshell python has found .pyd (shared lib), tried and failed to load it because a it can't find a dependant shared library.  Now the advice I have seen online regarding this is to dig out trusty dependancy walker, load the .pyd and see which dependencies the system can't find.

Now this is solid advice if we had a C++ application and/or we were using python < 3.8 (which uses the system/user path for dll re), however we are not, so even if dependency walker can't detect any problems we may still be facing the above error.

So what to do, well quick a quick and dirty solution is, if you are pretty sure of the folders containing your missing dll's, just add them to the dll search path in python and see if this reolves your issue.  For example if you have built with CUDA 
then unless you are using something like annaconda, you will need to add the directory containing opencv and hash (bin) and the directory containing the Nvidia shared libs.


you are most likely facing this error becuase the dll's inside C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/vMAJOR.MINOR/bin cannot be found, so you could try

In [5]:
import os;
os.add_dll_directory(OPENCV_BUID_PATH + "/bin/");
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");
os.add_dll_directory("D:\\3rd_party\\deps\\ffmpeg-n5.0-latest-win64-gpl-shared-5.0\\bin")
import cv2

In my case this solved the problem but what do you do if you don't know exactly what dll's are missing, well thanks to alec suggestion you can use process monitor https://docs.microsoft.com/en-us/sysinternals/downloads/procmon on windows to track down the problem.  With process monitor you can track the files are opened on the call to import cv2 and see exactly which dll can't be found.





In [ ]:
sys.

### Using Process Monitor fo find the missing dll

First download/extract/run process monitor. On opening you will be presented with the option to filter the output.  We only want to see files which are accesses by the python.exe process, therefore the filter options are

On the dropdown select "Process Name" and in the text box type python.exe and click Add - Process explorer will by defualt output for all processes this will reduce this down to just python.  

Then "Operation" is CreateFile -> this 

Process monitor produces a lot of output, these settings should filter for 

To make things easy to read make sure no other python process are running.

![title](imgs/proc_mon_filter.png)

Now start python and before typing import cv2, press the clear button (red trash can) in process monitor to clear any output generated during pythons initialization.

![title](imgs/proc_mon_failed_search_opencv.png)

The output shows that we successfully found cv2.cp..... which is why were not seeing ModuleNotFoundError: No module named 'cv2' however it also shows several attempts have been made to locate opencv_img_hash_460.dll and opencv_world460.dll without success even though they are on the search path.  Since Python 3.8.... 
Therefore we add

In [6]:
import os;
os.add_dll_directory(OPENCV_BUID_PATH + "/bin/");

In [7]:
import cv2

ImportError: DLL load failed while importing cv2: The specified module could not be found.

![title](imgs/proc_mon_failed_search_nvidia.png)

Now can see that opencv_img_hash_460.dll and opencv_world460.dll after a few attempts were located successfully however we are missing nppc64_11.dll etc. which are all part of the CUDA toolkit.  The solution add these as well and

In [10]:
import os;
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.7\\bin");
os.add_dll_directory("D:\\3rd_party\\deps\\ffmpeg-n5.0-latest-win64-gpl-shared-5.0\\bin")
import cv2

and in my case no more errors.  At this point if I was still seeing ImportError: DLL load failed while importing cv2: The specified module could not be found I would continue examining the output of process monitor until I get Result SUCCESS for all the .dll files python.exe tries to locate.

## Permanantely fixing

### Updating OpenCV's installed python loader

When OpenCV was installed .... only downside is that if you build install OpenCV again your changes will be lost

### Creating python loader

If you have installed the python bindings already this will feel a little bit like re-inventing the wheel, but it has the advantage that you can quickly modify a single file to point to access python bindings for a different OpenCV build without too much trouble and your configuration shouldn't get overidden

Recursion detected, when python .pyd is missing, check new install includes it, if not copy to lib/python to cv2/python-3.9 directory
How to init
